<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/Copy_of_Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install censusgeocode
!pip install datatable
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session
import datetime
from datetime import datetime,timedelta
!pip install pytz
import pytz
import datatable as dt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()


True

In [ ]:
import datetime
n_days_ago = datetime.date.today() - datetime.timedelta(days=180)
dt_string = n_days_ago.strftime("%Y-%m-%dT%H:%M:%S.000")

dt_string

In [ ]:
ListingId_result = rets_client.search(resource = 'Property', resource_class='Property', limit = 400,dmql_query='(StatusChangeTimestamp = {}+),(MlsStatus = SLD)'.format(dt_string))
lis = []
di ={}
df = pd.DataFrame()
for item in ListingId_result:
  if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence' and item['StateOrProvince']== 'Texas' and item['BedroomsTotal'] == '4' or item['BedroomsTotal'] == '3'):
   
    df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) 
    di = item   
for key in di:
  lis.append(key)

df = df.T
df.columns = lis
df['new_col'] = df['SubdivisionName'].astype(str).str[0:10]


n = len(pd.unique(df['new_col']))
result = df[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','LotSizeAcres','new_col']]  
n = df.groupby(by=['new_col'])["new_col"].count() 
n_by_state = df.groupby(by=['new_col','BedroomsTotal'])["ClosePrice"].sum() 
print(n.head(10))
print(n_by_state.head(10))
result.to_csv('SLD.csv')
#print(di)


new_col
Athens        1
Bayside       4
Brentwood     1
Charles Do    1
Dimpkins      1
ELLERBE RO    1
Edgemont P    1
H.C. Gresh    1
Harbor  Vi    1
Harbor Vil    2
Name: new_col, dtype: int64
new_col     BedroomsTotal
Athens      3                                          2200000.00
Bayside     3                441833.00386575.00396488.00416010.00
Brentwood   3                                           455400.00
Charles Do  3                                           365000.00
Dimpkins    3                                           134000.00
ELLERBE RO  3                                           180000.00
Edgemont P  3                                           120000.00
H.C. Gresh  3                                           183000.00
Harbor  Vi  3                                           449000.00
Harbor Vil  3                                  425000.00459000.00
Name: ClosePrice, dtype: object
